In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bile_Duct_Cancer"
cohort = "GSE131027"

# Input paths
in_trait_dir = "../../input/GEO/Bile_Duct_Cancer"
in_cohort_dir = "../../input/GEO/Bile_Duct_Cancer/GSE131027"

# Output paths
out_data_file = "../../output/preprocess/Bile_Duct_Cancer/GSE131027.csv"
out_gene_data_file = "../../output/preprocess/Bile_Duct_Cancer/gene_data/GSE131027.csv"
out_clinical_data_file = "../../output/preprocess/Bile_Duct_Cancer/clinical_data/GSE131027.csv"
json_path = "../../output/preprocess/Bile_Duct_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"High frequency of pathogenic germline variants in genes associated with homologous recombination repair in patients with advanced solid cancers"
!Series_summary	"We identified pathogenic and likely pathogenic variants in 17.8% of the patients within a wide range of cancer types. In particular, mesothelioma, ovarian cancer, cervical cancer, urothelial cancer, and cancer of unknown primary origin displayed high frequencies of pathogenic variants. In total, 22 BRCA1 and BRCA2 germline variant were identified in 12 different cancer types, of which 10 (45%) variants were not previously identified in these patients. Pathogenic germline variants were predominantly found in DNA repair pathways; approximately half of the variants were within genes involved in homologous recombination repair. Loss of heterozygosity and somatic second hits were identified in several of these genes, supporting possible causality for cancer development. A potential treatment t

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Analyze gene expression data availability
# Based on the background information, this dataset appears to be investigating
# expression features related to germline mutations in cancer patients.
# However, the data seems to be focused on classifying patients based on 
# mutation status and not providing actual gene expression data.
is_gene_available = False  # No clear gene expression matrix is visible in the output

# 2.1 Data Availability
# For trait (Bile_Duct_Cancer), we can extract from 'cancer' field at row 1
trait_row = 1

# No age information is available in the sample characteristics
age_row = None

# No gender information is available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert cancer type to binary (1 for bile duct cancer, 0 for others)"""
    if value is None:
        return None
    # Extract the value after colon and strip whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary based on our target trait
    if "Bile duct cancer" in value:
        return 1
    else:
        return 0

# Since age and gender are not available, we define empty conversion functions
def convert_age(value):
    """Placeholder function for age conversion"""
    return None

def convert_gender(value):
    """Placeholder function for gender conversion"""
    return None

# 3. Save metadata for initial filtering
# Trait data is available if trait_row is not None
is_trait_available = trait_row is not None

validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only execute if trait data is available
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # The structure should match what geo_select_clinical_features expects
    sample_char_dict = {
        0: ['tissue: tumor biopsy'], 
        1: ['cancer: Breast cancer', 'cancer: Colorectal cancer', 'cancer: Bile duct cancer', 
            'cancer: Mesothelioma', 'cancer: Urothelial cancer', 'cancer: Pancreatic cancer', 
            'cancer: Melanoma', 'cancer: Hepatocellular carcinoma', 'cancer: Ovarian cancer', 
            'cancer: Cervical cancer', 'cancer: Head and Neck cancer', 'cancer: Sarcoma', 
            'cancer: Prostate cancer', 'cancer: Adenoid cystic carcinoma', 'cancer: NSCLC', 
            'cancer: Oesophageal cancer', 'cancer: Thymoma', 'cancer: Others', 'cancer: CUP', 
            'cancer: Renal cell carcinoma', 'cancer: Gastric cancer', 'cancer: Neuroendocrine cancer', 
            'cancer: vulvovaginal'],
        2: ['mutated gene: ATR', 'mutated gene: FAN1', 'mutated gene: ERCC3', 'mutated gene: FANCD2', 
            'mutated gene: BAP1', 'mutated gene: DDB2', 'mutated gene: TP53', 'mutated gene: ATM', 
            'mutated gene: CHEK1', 'mutated gene: BRCA1', 'mutated gene: WRN', 'mutated gene: CHEK2', 
            'mutated gene: BRCA2', 'mutated gene: XPC', 'mutated gene: PALB2', 'mutated gene: ABRAXAS1', 
            'mutated gene: NBN', 'mutated gene: BLM', 'mutated gene: FAM111B', 'mutated gene: FANCA', 
            'mutated gene: MLH1', 'mutated gene: BRIP1', 'mutated gene: IPMK', 'mutated gene: RECQL', 
            'mutated gene: RAD50', 'mutated gene: FANCM', 'mutated gene: GALNT12', 'mutated gene: SMAD9', 
            'mutated gene: ERCC2', 'mutated gene: FANCC'],
        3: ['predicted: HRDEXP: HRD', 'predicted: HRDEXP: NO_HRD'],
        4: ['parp predicted: kmeans-2: PARP sensitive', 'parp predicted: kmeans-2: PARP insensitive']
    }
    
    # Convert the dictionary to a DataFrame
    # Create empty DataFrame with sample characteristics as rows
    rows = []
    # Getting the maximum number of unique values for any characteristic
    max_values = max(len(values) for values in sample_char_dict.values())
    
    # For each sample characteristic
    for row_idx, values in sample_char_dict.items():
        # Extend the list to match the maximum length with None values
        extended_values = values + [None] * (max_values - len(values))
        rows.append(extended_values)
    
    # Create DataFrame with rows as the characteristics and columns as the samples
    clinical_data = pd.DataFrame(rows)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted data
    preview = preview_df(selected_clinical_df)
    print("Preview of clinical data:", preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data: {0: [0.0], 1: [0.0], 2: [1.0], 3: [0.0], 4: [0.0], 5: [0.0], 6: [0.0], 7: [0.0], 8: [0.0], 9: [0.0], 10: [0.0], 11: [0.0], 12: [0.0], 13: [0.0], 14: [0.0], 15: [0.0], 16: [0.0], 17: [0.0], 18: [0.0], 19: [0.0], 20: [0.0], 21: [0.0], 22: [0.0], 23: [nan], 24: [nan], 25: [nan], 26: [nan], 27: [nan], 28: [nan], 29: [nan]}
Clinical data saved to ../../output/preprocess/Bile_Duct_Cancer/clinical_data/GSE131027.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be Affymetrix probe IDs, not standard human gene symbols.
# They follow Affymetrix's naming convention with numbers and suffixes like '_at', '_s_at', etc.
# These need to be mapped to human gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in the gene annotation dataframe contain the probe ID and gene symbol
prob_col = 'ID'  # This column contains probe IDs like '1007_s_at', '1053_at', etc.
gene_col = 'Gene Symbol'  # This column contains gene symbols like 'DDR1 /// MIR4640', 'RFC2', etc.

# 2. Get the gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Created mapping dataframe with {len(mapping_df)} probe-to-gene mappings")

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Converted data from {len(mapping_df)} probes to {len(gene_data)} genes")

# Preview the first few rows of the gene expression data
print("Gene expression data preview (first 5 rows, 5 columns):")
if len(gene_data.columns) > 5:
    print(gene_data.iloc[:5, :5])
else:
    print(gene_data.iloc[:5, :])


Created mapping dataframe with 45782 probe-to-gene mappings
Converted data from 45782 probes to 21278 genes
Gene expression data preview (first 5 rows, 5 columns):
          GSM3759992  GSM3759993  GSM3759994  GSM3759995  GSM3759996
Gene                                                                
A1BG        4.390919    9.637094    5.370776    7.376019    9.747455
A1BG-AS1    4.498580    4.911001    4.409248    4.958840    4.126732
A1CF        7.712909   17.768014    8.704946   14.905013   16.923252
A2M        14.491904   16.222561   15.166473   15.598188   15.317525
A2M-AS1     6.186831    4.286041    5.067774    5.807062    3.963854


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# First, normalize gene symbols using the function from the library
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Original gene count: {len(gene_data)}")
print(f"Normalized gene count: {len(normalized_gene_data)}")

# Create directory for the gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the normalized gene data to a CSV file
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. We need to first load or recreate the selected_clinical_df
try:
    # Try to load the previously saved clinical data
    selected_clinical_df = pd.read_csv(out_clinical_data_file)
    print("Loaded clinical data from file.")
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data sample:")
    print(selected_clinical_df.head())
except Exception as e:
    print(f"Error loading clinical data: {e}")
    # If loading fails, recreate the clinical features
    print("Recreating clinical features from raw data...")
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=True)
    print(f"Clinical data saved to {out_clinical_data_file}")

# Fix the clinical data format if necessary
# For geo_link_clinical_genetic_data, the clinical_df should be a DataFrame with
# trait and possibly age/gender as rows, and samples as columns
if trait not in selected_clinical_df.index and trait in selected_clinical_df.columns:
    print("Converting clinical data format - trait is in columns, needs to be in rows")
    selected_clinical_df = selected_clinical_df.T
    
print(f"Clinical data after formatting, shape: {selected_clinical_df.shape}")
print("First few sample IDs in clinical data:")
if hasattr(selected_clinical_df, 'columns'):
    print(list(selected_clinical_df.columns)[:5])
print("First few sample IDs in gene data:")
print(list(normalized_gene_data.columns)[:5])

# Check for column overlap
common_samples = set(selected_clinical_df.columns).intersection(set(normalized_gene_data.columns))
print(f"Number of common samples between clinical and gene data: {len(common_samples)}")

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# Check if linking was successful
if len(linked_data) == 0 or trait not in linked_data.columns:
    print("Linking clinical and genetic data failed - no valid rows or trait column missing")
    
    # Check what columns are in the linked data
    if len(linked_data.columns) > 0:
        print("Columns in linked data:")
        print(list(linked_data.columns)[:10])  # Print first 10 columns
    
    # Set is_usable to False and save cohort info
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=True,  # Consider it biased if linking fails
        df=pd.DataFrame({trait: [], 'Gender': []}), 
        note="Data linking failed - unable to process gene expression data appropriately."
    )
    print("The dataset was determined to be not usable for analysis.")
else:
    # 3. Handle missing values in the linked data
    linked_data = handle_missing_values(linked_data, trait)
    
    print(f"Linked data shape after handling missing values: {linked_data.shape}")
    
    # 4. Determine whether the trait and demographic features are severely biased
    trait_type = 'binary' if len(linked_data[trait].unique()) <= 2 else 'continuous'
    if trait_type == 'binary':
        if len(linked_data[trait].value_counts()) >= 2:
            is_trait_biased = judge_binary_variable_biased(linked_data, trait)
        else:
            print(f"Trait '{trait}' has only one unique value, considering it biased.")
            is_trait_biased = True
    else:
        is_trait_biased = judge_continuous_variable_biased(linked_data, trait)
    
    # Remove biased demographic features
    unbiased_linked_data = linked_data.copy()
    if 'Age' in unbiased_linked_data.columns:
        age_biased = judge_continuous_variable_biased(unbiased_linked_data, 'Age')
        if age_biased:
            print(f"The distribution of the feature \'Age\' in this dataset is severely biased.")
            unbiased_linked_data = unbiased_linked_data.drop(columns=['Age'])
    
    if 'Gender' in unbiased_linked_data.columns:
        if len(unbiased_linked_data['Gender'].value_counts()) >= 2:
            gender_biased = judge_binary_variable_biased(unbiased_linked_data, 'Gender')
            if gender_biased:
                print(f"The distribution of the feature \'Gender\' in this dataset is severely biased.")
                unbiased_linked_data = unbiased_linked_data.drop(columns=['Gender'])
        else:
            print(f"Gender has only one unique value, considering it biased and removing.")
            unbiased_linked_data = unbiased_linked_data.drop(columns=['Gender'])
    
    # 5. Conduct quality check and save the cohort information.
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=is_trait_biased, 
        df=unbiased_linked_data, 
        note="Dataset contains gene expression data from whole blood of systemic juvenile idiopathic arthritis (SJIA) patients treated with canakinumab or placebo and healthy controls."
    )
    
    # 6. If the linked data is usable, save it as a CSV file.
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("The dataset was determined to be not usable for analysis due to bias in the trait distribution.")

Original gene count: 21278
Normalized gene count: 19845


Normalized gene data saved to ../../output/preprocess/Bile_Duct_Cancer/gene_data/GSE131027.csv
Loaded clinical data from file.
Clinical data shape: (1, 30)
Clinical data sample:
     0    1    2    3    4    5    6    7    8    9  ...   20   21   22  23  \
0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0 NaN   

   24  25  26  27  28  29  
0 NaN NaN NaN NaN NaN NaN  

[1 rows x 30 columns]
Clinical data after formatting, shape: (1, 30)
First few sample IDs in clinical data:
['0', '1', '2', '3', '4']
First few sample IDs in gene data:
['GSM3759992', 'GSM3759993', 'GSM3759994', 'GSM3759995', 'GSM3759996']
Number of common samples between clinical and gene data: 0
Linked data shape: (122, 19846)
Linking clinical and genetic data failed - no valid rows or trait column missing
Columns in linked data:
[0, 'A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT']
Abnormality detected in the cohort: GSE131027. Preprocessing failed.
The dataset wa